In [1]:
import pandas as pd             
import matplotlib.pyplot as plt 
import datetime as dt
import numpy as np

# these are new 
import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import shutil                   # file management tools 
import os                       # operating system tools (check files)

from census import Census # This is new...

import geopandas as gpd # this is the main geopandas 
from shapely.geometry import Point, Polygon # also needed

import pyarrow as pa
import pyarrow.parquet as pq


from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

## Interactive Covid-19 map by state  

---

### Step \#1. Grab data from https://github.com/camckenzie/COVID-19-Interactive-Map



In [2]:
col_list = ['State/Territory',
        'Total Cases', 'Confirmed Cases', 'Cases in Last 7 Days',
        'Case Rate per 100000', 'Total Deaths']

df = pd.read_csv("https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/covid19_cases.csv", skiprows=3, usecols=col_list)

In [42]:
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "New York State", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

df = df[df["State/Territory"].isin(state_names)]
df = df.reset_index(drop=True)

total_cases = df['Total Cases'].sum()
df

,State/Territory,Total Cases,Confirmed Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location
0,Alaska,60333,0,950,8247,309,615825,84181.0,411529,56255.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
1,Alabama,515388,401490,2677,10511,10554,2839540,57912.0,1771979,36139.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
2,Arkansas,330398,0,1221,10948,5626,1814280,60119.0,1209965,40094.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
3,Arizona,841811,782367,3962,11565,16967,4365955,59982.0,3512579,48258.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
4,California,3568426,0,17370,9031,57936,24288100,61470.0,18604164,47085.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
5,Colorado,462081,433517,8561,8024,6107,3459750,60078.0,2746030,47685.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
6,Connecticut,310888,287235,8866,8720,7886,2484595,69688.0,1992899,55897.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
7,Delaware,94802,89249,1983,9736,1559,621905,63866.0,473912,48668.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
8,Florida,2019500,0,35226,9403,33425,13261815,61747.0,9781739,45544.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
9,Georgia,1059548,852395,10151,9979,19055,6101935,57471.0,3950645,37209.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...


In [4]:
col_list2 = ['State/Territory/Federal Entity',
        'Total Doses Delivered', 'Doses Delivered per 100K',
        'Total Doses Administered by State where Administered', 'Doses Administered per 100k by State where Administered']

df1 = pd.read_csv("https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/covid19_vaccinations.csv", skiprows=3, usecols=col_list2)

In [43]:
df1.rename(columns={'State/Territory/Federal Entity' : "State/Territory",
                    'Total Doses Administered by State where Administered' :
                    'Total Doses Administered',
                    'Doses Administered per 100k by State where Administered' : 
                    'Doses Administered per 100k'}, inplace=True)

df1 = df1[df1["State/Territory"].isin(state_names)]
df1 = df1.reset_index(drop=True)

total_vaccines = df1["Total Doses Delivered"].sum()
df1

,State/Territory,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k
0,Alaska,615825,84181.0,411529,56255.0
1,Alabama,2839540,57912.0,1771979,36139.0
2,Arkansas,1814280,60119.0,1209965,40094.0
3,Arizona,4365955,59982.0,3512579,48258.0
4,California,24288100,61470.0,18604164,47085.0
5,Colorado,3459750,60078.0,2746030,47685.0
6,Connecticut,2484595,69688.0,1992899,55897.0
7,Delaware,621905,63866.0,473912,48668.0
8,Florida,13261815,61747.0,9781739,45544.0
9,Georgia,6101935,57471.0,3950645,37209.0


In [6]:
df = df.join(df1["Total Doses Delivered"])
df = df.join(df1["Doses Delivered per 100K"])
df = df.join(df1["Total Doses Administered"])
df = df.join(df1["Doses Administered per 100k"])

df

,State/Territory,Total Cases,Confirmed Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k
0,Alaska,60333,0,950,8247,309,615825,84181.0,411529,56255.0
1,Alabama,515388,401490,2677,10511,10554,2839540,57912.0,1771979,36139.0
2,Arkansas,330398,0,1221,10948,5626,1814280,60119.0,1209965,40094.0
3,Arizona,841811,782367,3962,11565,16967,4365955,59982.0,3512579,48258.0
4,California,3568426,0,17370,9031,57936,24288100,61470.0,18604164,47085.0
5,Colorado,462081,433517,8561,8024,6107,3459750,60078.0,2746030,47685.0
6,Connecticut,310888,287235,8866,8720,7886,2484595,69688.0,1992899,55897.0
7,Delaware,94802,89249,1983,9736,1559,621905,63866.0,473912,48668.0
8,Florida,2019500,0,35226,9403,33425,13261815,61747.0,9781739,45544.0
9,Georgia,1059548,852395,10151,9979,19055,6101935,57471.0,3950645,37209.0


In [7]:
file_path = os.getcwd()

doc_path = file_path + "\\Docs\\"

html_path = "https://github.com/camckenzie/COVID-19-Interactive-Map/tree/main/Docs/Cases_PNGs"

df["cases_file_location"] = html_path + "state_" + ".png"

html_path = "https://github.com/camckenzie/COVID-19-Interactive-Map/tree/main/Docs/Vaccinations_PNGs"

df["vaccine_file_location"] = html_path + "state_" + ".png"

In [8]:
df.head()

,State/Territory,Total Cases,Confirmed Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location
0,Alaska,60333,0,950,8247,309,615825,84181.0,411529,56255.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
1,Alabama,515388,401490,2677,10511,10554,2839540,57912.0,1771979,36139.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
2,Arkansas,330398,0,1221,10948,5626,1814280,60119.0,1209965,40094.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
3,Arizona,841811,782367,3962,11565,16967,4365955,59982.0,3512579,48258.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...
4,California,3568426,0,17370,9031,57936,24288100,61470.0,18604164,47085.0,https://github.com/camckenzie/COVID-19-Interac...,https://github.com/camckenzie/COVID-19-Interac...


In [9]:
map_projection = "epsg:2163"

In [10]:
cwd = os.getcwd()

lake_shape = cwd + "\\shapefiles\\lake\\ne_10m_lakes.shx"

lake_map = gpd.read_file(lake_shape)

lake_map = lake_map.to_crs({'init': map_projection})

C:\Users\Andrew\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
cwd = os.getcwd()

land_shape = cwd + "\\shapefiles\\land\\ne_50m_land.shx"

land_map = gpd.read_file(land_shape)

land_map = land_map.to_crs({'init': map_projection})

land_map = land_map.iloc[0:1200]

In [12]:
cwd = os.getcwd()

state_shape = cwd + "\\shapefiles\\state\\tl_2017_us_state.shx"

us_map = gpd.read_file(state_shape)

us_map = us_map.to_crs({'init': map_projection})

In [13]:
us_map["geometry"] = us_map["geometry"].simplify(200)

In [14]:
us_map["area_fips"] = (us_map.STATEFP.astype(str)).astype(int)

---

###  Step \#2 Merge and Clean

The next couple of cells download the requisite shapefiles from the US census. They are unzipped in a folder called shapefiles and then county. So they are assuming some structure behind your folder setup. 

In [15]:
us_map.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,area_fips
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62265662566,489840834,+38.6472854,-080.6183274,"POLYGON ((1561571.961 -483477.971, 1561616.796...",54
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138911437206,31398800291,+28.4574302,-082.4091478,"MULTIPOLYGON (((1734277.712 -2086763.305, 1734...",12
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143784114293,6211277447,+40.1028754,-089.1526108,"POLYGON ((742564.486 -445238.183, 746260.695 -...",17
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206229176104,18944967530,+46.3158148,-094.1996628,"POLYGON ((245479.802 186204.457, 244632.514 18...",27
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25150696145,6980371026,+38.9466584,-076.6744939,"POLYGON ((1914704.978 -379436.546, 1914436.356...",24


In [19]:
us_map = us_map.merge(df, left_on='area_fips', right_on='Total Cases', how="left", indicator=True)

In [20]:
us_map.set_index("STATEFP", inplace = True)

drop_list = ["72","78","69","66","60","11"]

us_map.drop(drop_list, inplace = True)
us_map.reset_index()
us_map

,REGION,DIVISION,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,...,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location,_merge
STATEFP,,,,,,,,,,,,,,,,,,,,,
54,3,5,01779805,54,WV,West Virginia,00,G4000,A,62265662566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
12,3,5,00294478,12,FL,Florida,00,G4000,A,138911437206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
17,2,3,01779784,17,IL,Illinois,00,G4000,A,143784114293,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
27,2,4,00662849,27,MN,Minnesota,00,G4000,A,206229176104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
24,3,5,01714934,24,MD,Maryland,00,G4000,A,25150696145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
44,1,1,01219835,44,RI,Rhode Island,00,G4000,A,2677997539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
16,4,8,01779783,16,ID,Idaho,00,G4000,A,214048160737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
33,1,1,01779794,33,NH,New Hampshire,00,G4000,A,23187445452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
37,3,5,01027616,37,NC,North Carolina,00,G4000,A,125919712692,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [26]:
us_map["cases_label"] = us_map["Total Cases"].round(0)
us_map["cases_label"] = us_map["cases_label"].map('{:,.0f}'.format)
us_map["vaccines_label"] = us_map["Total Doses Delivered"].round(0)
us_map["vaccines_label"] = us_map["vaccines_label"].map('{:,.0f}'.format)

In [27]:
my_api_key = 'c3f6481fd1b41113279e1e9cb70099e28032d3b4'

c = Census(my_api_key)

code = ("NAME","B01001_001E") 

state_2020 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'state:*'}, year=2019))
        
state_2020 = state_2020.rename(columns = {"B01001_001E":"2020_population"})

state_2020["GEOFIPS"] = (state_2020["state"]).astype(int)

state_2020["2020_population"] = state_2020["2020_population"].astype(float)

state_2020.set_index(["GEOFIPS"], inplace = True)
state_2020.head()

,NAME,2020_population,state
GEOFIPS,,,
1,Alabama,4876250.0,01
2,Alaska,737068.0,02
4,Arizona,7050299.0,04
5,Arkansas,2999370.0,05
6,California,39283497.0,06


In [28]:
state_2020 = state_2020[state_2020["NAME"].isin(state_names)]
#state_2020 = state_2020.reset_index(drop=True)
#state_2020.set_index(["GEOFIPS"], inplace = True)

state_2020.head()

,NAME,2020_population,state
GEOFIPS,,,
1,Alabama,4876250.0,01
2,Alaska,737068.0,02
4,Arizona,7050299.0,04
5,Arkansas,2999370.0,05
6,California,39283497.0,06


In [29]:
us_map = us_map.merge(state_2020[["2020_population"]], left_on='area_fips', right_on="GEOFIPS", how="left")

In [30]:
us_map['pop_label'] = us_map["2020_population"].map('{:,.0f}'.format)

In [34]:
q_cases = [0,1,5,10,100,250,500,1000,5000,10000,np.inf]

us_map["q_cases"]= pd.cut(us_map["Total Cases"],q_cases, labels=range(0,10))
us_map["q_vaccines"]= pd.cut(us_map["Total Doses Delivered"],q_cases, labels=range(0,10))

us_map["q_cases"].fillna(0, inplace = True)
us_map["q_vaccines"].fillna(0, inplace = True)

us_map["cases_label"].replace("nan", "N.R.", inplace=True)
us_map["vaccines_label"].replace("nan", "N.R.", inplace=True)
us_map

,REGION,DIVISION,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,...,q_cases,q_vaccines,cases_label,vaccines_label,2020_population,pop_label,scalerank,featurecla,min_zoom,geometry
0,3,5,01779805,54,WV,West Virginia,00,G4000,A,62265662566,...,0,0,N.R.,N.R.,1817305.0,"1,817,305",0,Land,0.0,"POLYGON ((1561616.796 -481648.001, 1560757.869..."
1,3,5,00294478,12,FL,Florida,00,G4000,A,138911437206,...,0,0,N.R.,N.R.,20901636.0,"20,901,636",0,Land,0.0,"POLYGON ((1347348.672 -1442958.208, 1431360.07..."
2,2,3,01779784,17,IL,Illinois,00,G4000,A,143784114293,...,0,0,N.R.,N.R.,12770631.0,"12,770,631",0,Land,0.0,"POLYGON ((747568.711 -440784.746, 748047.286 -..."
3,2,4,00662849,27,MN,Minnesota,00,G4000,A,206229176104,...,0,0,N.R.,N.R.,5563378.0,"5,563,378",0,Land,0.0,"MULTIPOLYGON (((244436.569 186031.710, 245325...."
4,3,5,01714934,24,MD,Maryland,00,G4000,A,25150696145,...,0,0,N.R.,N.R.,6018848.0,"6,018,848",0,Land,0.0,"MULTIPOLYGON (((1906445.142 -376340.044, 19044..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,4,9,01785533,02,AK,Alaska,00,G4000,A,1478588231566,...,0,0,N.R.,N.R.,737068.0,"737,068",0,Land,3.0,"POLYGON ((-1959870.825 1676823.850, -1962422.3..."
169,4,9,01785533,02,AK,Alaska,00,G4000,A,1478588231566,...,0,0,N.R.,N.R.,737068.0,"737,068",0,Land,3.0,"POLYGON ((-1937369.478 1912679.707, -1944243.1..."
170,3,6,01779790,28,MS,Mississippi,00,G4000,A,121530256608,...,0,0,N.R.,N.R.,2984418.0,"2,984,418",0,Land,4.0,"POLYGON ((1103633.318 -1567567.827, 1094882.49..."
171,1,1,00606926,25,MA,Massachusetts,00,G4000,A,20204287539,...,0,0,N.R.,N.R.,6850553.0,"6,850,553",0,Land,3.0,"POLYGON ((2376854.443 22635.532, 2372284.419 2..."


In [32]:
us_map = gpd.overlay(us_map, land_map, how='intersection')

In [33]:
great_lakes = ["Lake Superior", "Lake Michigan", "Lake Erie", "Lake Superior", "Lake Huron"]
us_map = gpd.overlay(us_map, lake_map[lake_map.name.isin(great_lakes)], how='difference')

In [35]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)

from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.models import Title
from bokeh.layouts import gridplot

from bokeh.plotting import figure, save
from bokeh.models import Panel, Tabs

from bokeh.resources import CDN
from bokeh.embed import file_html

from bokeh.io import output_file, show
from bokeh.models import Div

In [37]:
state_geosource = GeoJSONDataSource(geojson = us_map.to_json())

palette = brewer['YlOrRd'][4]

color_mapper = LinearColorMapper(palette = palette, low=0, high=9)
tick_labels = {2:str(q_cases[2]), 4:str(q_cases[4]), 6:str(q_cases[6]), 8:str(q_cases[9])}

In [44]:
title = "Covid-19 Cases by State" + ", Total Cases: " + f"{total_cases:,d}"

color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0), major_label_overrides = tick_labels,
                     major_tick_line_alpha = .25)

# Create figure object.
p = figure( 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "box_zoom, reset")

descip = "Colorbar by # of covid-19 cases; Hover tool plots cases by day."
descip = descip + "Data from https://covid.cdc.gov/covid-data-tracker/#cases_totalcases"
p.add_layout(Title(text=descip, text_font_style="italic", text_font_size="9pt"), 'above')
p.add_layout(Title(text=title, text_font_size="11pt"), 'above')

author = "Created by Andrew Chuah"
p.add_layout(Title(text=author, text_font_style="italic", text_font_size="9pt"), 'below')

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Add patch renderer to figure.

states = p.patches('xs','ys', source = state_geosource,
                   fill_color = {"field" :'q_cases',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)


TOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95;">
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold;">@NAME
        </div>
        <div>
            <img
                src="@cases_file_location" height="200" alt="@cases_file_location" width="350"
                style="float: center; margin: 1px 1px 1px 1px; opacity: 0.95;"
                border="0"
            ></img>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold">Population: @pop_label &nbsp &nbsp Cases: @cases_label &nbsp &nbsp 
            Deaths: @deaths_label</span>
        </div>
    </div>
"""

p.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))

In [46]:
title_vaccines = "Covid-19 Vaccinations by State" + ", Total Doses Delivered: " + f"{total_vaccines:,d}"

color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0), major_label_overrides = tick_labels,
                     major_tick_line_alpha = .25)

# Create figure object.
pdth = figure( 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "box_zoom, reset")

descip = "Colorbar by # of covid-19 cases; Hover tool plots cases by day."
descip = descip + " Data from https://covid.cdc.gov/covid-data-tracker/#cases_totalcases"
pdth.add_layout(Title(text=descip, text_font_style="italic", text_font_size="9pt"), 'above')
pdth.add_layout(Title(text=title_vaccines, text_font_size="11pt"), 'above')

author = "Created by Andrew Chuah"
pdth.add_layout(Title(text=author, text_font_style="italic", text_font_size="9pt"), 'below')

pdth.xgrid.grid_line_color = None
pdth.ygrid.grid_line_color = None
# Add patch renderer to figure.

states = pdth.patches('xs','ys', source = state_geosource,
                   fill_color = {"field" :'q_vaccines',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

TOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95;">
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold;">@NAME
        </div>
        <div>
            <img
                src="@deaths_file_location" height="200" alt="@deaths_file_location" width="350"
                style="float: center; margin: 1px 1px 1px 1px; opacity: 0.95;"
                border="0"
            ></img>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold">Population: @pop_label &nbsp &nbsp Cases: @cases_label &nbsp &nbsp 
            Deaths: @deaths_label</span>
        </div>
    </div>
"""

pdth.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))

In [47]:
tab1 = Panel(child=p, title="Cases")
tab2 = Panel(child=pdth, title="Vaccines")

tabs = Tabs(tabs=[ tab1, tab2 ])

file_path = os.getcwd()

doc_path = file_path +"\\Docs"

outfp = doc_path + "\\us_covid_map.html"

# Save the map
save(tabs, outfp)

# Not sure if this is important, but seemed to start working once
# I ran it
html = file_html(tabs, CDN, outfp)

C:\Users\Andrew\anaconda3\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\Andrew\anaconda3\lib\site-packages\bokeh\io\saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
